In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW ticker_value_update AS
SELECT 
  tv.Ticker, CAST(tv.variable as DATE) as stock_date, Open, Close, High, Low, Volume
FROM 
  bronze.ticker_value tv
PIVOT (
  MAX(CAST(tv.value AS DECIMAL(20,2)))
  FOR Value_Type IN ('1. open' AS Open, '4. close' AS Close, '2. high' as High, '3. low' as Low, '5. volume' as Volume)
)
LEFT JOIN (SELECT MIN(CAST(variable AS DATE)) AS min_variable, Ticker 
          FROM 
            bronze.ticker_value 
          WHERE 
            value IS NOT NULL GROUP BY Ticker) tv1
  ON tv.Ticker = tv1.Ticker
WHERE 
  CAST(variable as DATE) > CAST(tv1.min_variable AS DATE)

In [0]:
%sql
MERGE INTO silver.s_ticker_value
USING ticker_value_update
ON silver.s_ticker_value.Ticker=ticker_value_update.Ticker
and silver.s_ticker_value.stock_date = ticker_value_update.stock_date
WHEN MATCHED THEN
UPDATE SET *
WHEN NOT MATCHED THEN INSERT *

In [0]:
%python

sp_silver_ticker_metadata = spark.sql('''
 SELECT
    tm.*, tl.name, tl.`exchange`, tl.assetType
FROM
  bronze.ticker_metadata tm

LEFT JOIN bronze.ticker_listing tl
    ON tm.Symbol = tl.Symbol
''')


sp_silver_ticker_metadata.distinct().createOrReplaceTempView('temp_sp_silver_ticker_metadata')
spark.sql('''
CREATE TABLE IF NOT EXISTS silver.s_ticker_metadata
USING DELTA
AS SELECT * FROM temp_sp_silver_ticker_metadata
''')

In [0]:
%sql
MERGE INTO silver.s_ticker_metadata
USING temp_sp_silver_ticker_metadata
ON silver.s_ticker_metadata.Ticker=temp_sp_silver_ticker_metadata.Ticker
WHEN MATCHED THEN
UPDATE SET *
WHEN NOT MATCHED THEN INSERT *

In [0]:
%sql

select * from silver.s_ticker_value